In [1]:
import tensorflow as tf

C:\Users\jamie\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras

Using TensorFlow backend.


In [3]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [4]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
from dateutil.parser import parse
from itertools import chain
import operator
import sys
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [5]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    df.head()
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [6]:
def plot(actual,predict,real_data,columns,title):
# lot_data(predicted, actual, future, cols,title)
    df = pd.DataFrame(real_data)
    data_series = df["Date"]
    # print("Here")
    # lot_data(predicted, actual, future, cols,title)
    df = pd.DataFrame(real_data)
    
    trace1 = go.Scatter(
        x=data_series,
        y=actual,
        name='Actual Close'
    )
    #Lower_closing  Upper_closing
    trace2 = go.Scatter(
        x=data_series,
        y=predict,
        name='Predicted Close',
        fill=None,
        mode='lines',
        line=dict(color='rgb(143,19,131)'),        
       
    )
    data = [trace1, trace2]
    layout = go.Layout(
        title='Actual Vs Predicted using Google TensorFlow RNN',
        yaxis=dict(
            title='Closing Price'
        ),
        yaxis2=dict(
            title='yaxis2 title',
            titlefont=dict(
                color='rgb(148, 103, 189)'
            ),
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            overlaying='y',
            side='right'
        )
    )
    graph = go.Figure(data=data, layout=layout)
    return graph

In [7]:
btc_data = pd.read_csv("data_final.csv")
btc_data.set_index('Date', inplace=True)
select_list = ["open","high","high_low_spread","effective_spread","7_trailing","7_next_trailing","volume_btc"]
btc_data.head()

,open,high,low,close,weighted_price,high_low_spread,midquote_price,effective_spread,7_trailing,7_next_trailing,volume_btc,volume_currency
Date,,,,,,,,,,,,
2016-01-01,431.35,435.88,428.22,433.66,432.87,7.66,3.83,859.65,-22.526170,20.216172,8749.673805,3789078.83
2016-01-02,434.71,435.99,431.18,434.44,433.70,4.81,2.41,864.07,16.345000,15.412500,6591.310053,2862465.49
2016-01-03,434.02,434.53,424.06,429.88,429.01,10.47,5.24,849.28,5.965002,19.124997,9678.402186,4157151.63
2016-01-04,430.43,435.12,428.82,433.01,432.40,6.30,3.15,859.72,10.813537,16.589110,13463.971066,5832898.79
2016-01-05,432.75,435.14,429.50,432.35,432.15,5.64,2.82,859.06,0.085000,3.425000,10381.184703,4492484.21


In [8]:
test_list = ['open','high','close','weighted_price','7_trailing','midquote_price','volume_btc']
btc_data = btc_data[test_list]
btc_data.head()

,open,high,close,weighted_price,7_trailing,midquote_price,volume_btc
Date,,,,,,,
2016-01-01,431.35,435.88,433.66,432.87,-22.526170,3.83,8749.673805
2016-01-02,434.71,435.99,434.44,433.70,16.345000,2.41,6591.310053
2016-01-03,434.02,434.53,429.88,429.01,5.965002,5.24,9678.402186
2016-01-04,430.43,435.12,433.01,432.40,10.813537,3.15,13463.971066
2016-01-05,432.75,435.14,432.35,432.15,0.085000,2.82,10381.184703


In [9]:
close_var = "var{}(t)".format(list(btc_data.columns.values).index('close')+1)
close_var

'var3(t)'

In [10]:
values = btc_data.values
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

reframed = series_to_supervised(scaled, 1, 1)
for var_name in reframed.columns.values:
    if "(t)" in var_name and var_name != close_var:
        reframed.drop(var_name, axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var3(t)
1,0.008129,0.008004,0.008256,0.008176,0.178082,0.001033,0.038771,0.008297
2,0.008305,0.008010,0.008297,0.008220,0.498630,0.000434,0.026336,0.008059
3,0.008269,0.007935,0.008059,0.007973,0.416438,0.001628,0.044121,0.008222
4,0.008081,0.007965,0.008222,0.008152,0.465753,0.000746,0.065931,0.008188
5,0.008202,0.007966,0.008188,0.008138,0.341096,0.000607,0.048170,0.008060


In [11]:
timeseries_values = reframed.values
n_train_hours = 650
train = timeseries_values[:n_train_hours, :]
test = timeseries_values[n_train_hours:, :]
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
train_X_orig, train_y_orig = train[:, :-1], train[:, -1]
test_X_orig, test_y_orig = test[:, :-1], test[:, -1]

In [12]:
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam') #mae
### fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Train on 650 samples, validate on 80 samples
Epoch 1/50
 - 2s - loss: 0.0040 - val_loss: 0.2433
Epoch 2/50
 - 0s - loss: 0.0025 - val_loss: 0.1954
Epoch 3/50
 - 0s - loss: 0.0025 - val_loss: 0.1623
Epoch 4/50
 - 0s - loss: 0.0028 - val_loss: 0.1409
Epoch 5/50
 - 0s - loss: 0.0027 - val_loss: 0.1263
Epoch 6/50
 - 0s - loss: 0.0025 - val_loss: 0.1143
Epoch 7/50
 - 0s - loss: 0.0022 - val_loss: 0.1026
Epoch 8/50
 - 0s - loss: 0.0019 - val_loss: 0.0907
Epoch 9/50
 - 0s - loss: 0.0017 - val_loss: 0.0788
Epoch 10/50
 - 0s - loss: 0.0015 - val_loss: 0.0675
Epoch 11/50
 - 0s - loss: 0.0013 - val_loss: 0.0571
Epoch 12/50
 - 0s - loss: 0.0011 - val_loss: 0.0478
Epoch 13/50
 - 0s - loss: 9.9633e-04 - val_loss: 0.0394
Epoch 14/50
 - 0s - loss: 8.6173e-04 - val_loss: 0.0320
Epoch 15/50
 - 0s - loss: 7.4017e-04 - val_loss: 0.0256
Epoch 16/50
 - 0s - loss: 6.3257e-04 - val_loss: 0.0200
Epoch 17/50
 - 0s - loss: 5.3859e-04 - val_loss: 0.0154
Epoch 18/50
 - 0s - loss: 4.5752e-04 - val_loss: 0.0116
Epoc

In [13]:
ypred = model.predict(test_X)
test_X_revert = test_X.reshape((test_X.shape[0], test_X.shape[2]))
inv_ypred = np.concatenate((ypred, test_X_revert[:, 1:]), axis=1)
inv_ypred = scaler.inverse_transform(inv_ypred)
inv_yhat = inv_ypred[:,0]

In [14]:
test_y_revert = test_y.reshape((len(test_y), 1))
inv_yorig = np.concatenate((test_y_revert, test_X_revert[:, 1:]), axis=1)
inv_yorig = scaler.inverse_transform(inv_yorig)
inv_y = inv_yorig[:,0]

In [15]:
cols = ["Mean", "Lower_closing", "Upper_closing"]
title = "Closing price distribution of bitcoin"
btc_data.reset_index(inplace=True)
real_data = btc_data[651:len(btc_data)]

In [16]:
real_data.head()

,Date,open,high,close,weighted_price,7_trailing,midquote_price,volume_btc
651,2017-10-13,5437.71,5859.46,5641.19,5625.06,702.0,237.53,65665.908165
652,2017-10-14,5640.85,5812.34,5802.14,5696.09,703.0,125.27,21269.396307
653,2017-10-15,5800.12,5837.09,5689.60,5616.58,696.0,198.49,28901.473032
654,2017-10-16,5690.57,5786.24,5746.26,5686.54,693.0,111.83,21918.467929
655,2017-10-17,5749.81,5767.68,5598.70,5616.65,686.0,122.60,21831.571158
